In [23]:
import os
current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_v3")
print(current_dir,persistent_directory)

c:\Users\user1\Documents\github_clone\ATx2025_chihlee_langchain\lesson7 c:\Users\user1\Documents\github_clone\ATx2025_chihlee_langchain\lesson7\db\chroma_db_v3


In [24]:
os.path.abspath("__file__")

'c:\\Users\\user1\\Documents\\github_clone\\ATx2025_chihlee_langchain\\lesson7\\__file__'

In [25]:
current_dir = os.path.dirname(os.path.abspath("__file__"))
current_dir

'c:\\Users\\user1\\Documents\\github_clone\\ATx2025_chihlee_langchain\\lesson7'

In [ ]:
import os
import shutil # 導入 shutil 模組

# 這裡假設你的原始程式碼意圖是找出包含這個腳本的目錄
# 不過 os.path.abspath("__file__") 在某些環境下（例如互動式 shell）可能行為不一致。
# 對於實際的腳本文件，應該使用 os.path.abspath(__file__)，
# 但因為你在 Jupyter/互動環境中提供 '__file__' 字串，
# 我將保留原來的邏輯，但為了安全起見，建議在實際腳本中用 __file__ (不帶引號)。

# 獲取當前腳本的目錄（根據你提供的原始程式碼邏輯）
# 在實際執行時，如果這是腳本文件，應使用 os.path.abspath(__file__)
# 這裡我們保留你原始程式碼的結構
current_dir = os.path.dirname(os.path.abspath("__file__"))

# 定義要刪除的目錄路徑
persistent_directory = os.path.join(current_dir, "db" )

print(f"嘗試刪除目錄: {persistent_directory}")

# 檢查目錄是否存在 , 遞迴刪除目錄及其所有內容
# if os.path.isdir(persistent_directory):
#     try:
#         # 使用 shutil.rmtree() 遞迴刪除目錄及其所有內容
#         shutil.rmtree(persistent_directory)
#         print(f"成功刪除目錄及其內容: {persistent_directory}")
#     except OSError as e:
#         print(f"錯誤: 刪除目錄 {persistent_directory} 失敗: {e}")
# else:
#     print(f"目錄不存在: {persistent_directory}")

# 如果你只想刪除 'db' 目錄內的**內容**，但保留 'db' 目錄本身，則程式碼會更複雜：
# if os.path.isdir(persistent_directory):
#     for filename in os.listdir(persistent_directory):
#         file_path = os.path.join(persistent_directory, filename)
#         try:
#             if os.path.isfile(file_path) or os.path.islink(file_path):
#                 os.unlink(file_path) # 刪除檔案或符號連結
#             elif os.path.isdir(file_path):
#                 shutil.rmtree(file_path) # 遞迴刪除子目錄
#         except Exception as e:
#             print(f'刪除 {file_path} 失敗. 原因: {e}')
#     print(f"成功清空目錄內容: {persistent_directory}")
# else:
#     print(f"目錄不存在: {persistent_directory}")

嘗試刪除目錄: c:\Users\user1\Documents\github_clone\ATx2025_chihlee_langchain\lesson7\db


In [27]:
import os

# 獲取當前腳本的目錄路徑
# 注意：在實際的 Python 腳本中，建議使用 os.path.abspath(__file__)
# 但因為您提供的原始碼使用了 "__file__" 字串，這裡保留原始邏輯。
current_dir = os.path.dirname(os.path.abspath("__file__"))

# 定義目標目錄路徑
persistent_directory = os.path.join(current_dir, "db" )

# 打印路徑，確認是我們想要操作的目錄
print(f"目標目錄路徑: {persistent_directory}")

# 檢查目錄是否存在，如果不存在則創建它。
# exist_ok=True 確保如果目錄已經存在，不會拋出 FileExistsError 錯誤。
# os.makedirs 會創建路徑中的所有中間目錄。
try:
    os.makedirs(persistent_directory, exist_ok=True)
    print(f"目錄已確認/創建: {persistent_directory}")
except OSError as e:
    # 捕獲其他可能的錯誤，例如權限問題
    print(f"創建目錄失敗: {e}")

目標目錄路徑: c:\Users\user1\Documents\github_clone\ATx2025_chihlee_langchain\lesson7\db
目錄已確認/創建: c:\Users\user1\Documents\github_clone\ATx2025_chihlee_langchain\lesson7\db


In [28]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb

current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_v3")
try:
    if os.path.exists(persistent_directory):
        loader = TextLoader(file_path=file_path,encoding='utf-8')
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)
        print("向量資料庫已存在")

    if not os.path.exists(persistent_directory):
        if not os.path.exists(file_path):
            raise FileNotFoundError(
                f"檔案{file_path}不存在,請檢查路徑"
            )

        loader = TextLoader(file_path=file_path,encoding='utf-8')
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        docs = text_splitter.split_documents(documents)
        #print(len(docs))
        #print(docs[0].page_content)
        embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3"
        )

        client = chromadb.PersistentClient(path=persistent_directory)

        db = Chroma.from_documents(
            docs,
            embedding=embeddings,
            client = client,
            collection_name="smartphone_manual"
        )

    else:
        print("向量資料庫已經存在,不需要初始化")
        
except Exception as error:
    print(error)


向量資料庫已存在
向量資料庫已經存在,不需要初始化


In [29]:
query = "手機硬體規格如何?"

embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3"
        )

client = chromadb.PersistentClient(path=persistent_directory)

db = Chroma.from_documents(
            docs,
            embedding=embeddings,
            client = client,
            collection_name="smartphone_manual"
        )

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

relevant_docs = retriever.invoke(query)
for doc in relevant_docs:
    print(doc.page_content)
    print("=" * 60)

8.4 聯絡客服
客服管道:
- 客服專線:0800-123-456(週一至週日 9:00-21:00)
- 線上客服:官網即時聊天
- 電子郵件:support@example.com
- 服務中心:請至官網查詢鄰近服務中心地址

送修前準備:
1. 備份所有資料
2. 移除 SIM 卡與記憶卡
3. 準備購買證明(發票或收據)
4. 記錄故障現象與發生時間

規格總覽

硬體規格:
- 處理器:Snapdragon 8 Gen 2 / Apple A17 Pro
- 記憶體:8GB/12GB RAM
- 儲存空間:128GB/256GB/512GB
- 螢幕:6.7" AMOLED,120Hz
- 電池:5000mAh
- 重量:195g
- 尺寸:163.1 x 75.5 x 8.2 mm

相機:
- 後置:50MP + 12MP + 8MP
- 前置:32MP

連線:
- 5G、4G LTE
- Wi-Fi 6、藍牙 5.3
- NFC、GPS
- USB-C

作業系統:
- Android 14 或 iOS 17
- 承諾 4 年系統更新
- 5 年安全性更新
8.4 聯絡客服
客服管道:
- 客服專線:0800-123-456(週一至週日 9:00-21:00)
- 線上客服:官網即時聊天
- 電子郵件:support@example.com
- 服務中心:請至官網查詢鄰近服務中心地址

送修前準備:
1. 備份所有資料
2. 移除 SIM 卡與記憶卡
3. 準備購買證明(發票或收據)
4. 記錄故障現象與發生時間

規格總覽

硬體規格:
- 處理器:Snapdragon 8 Gen 2 / Apple A17 Pro
- 記憶體:8GB/12GB RAM
- 儲存空間:128GB/256GB/512GB
- 螢幕:6.7" AMOLED,120Hz
- 電池:5000mAh
- 重量:195g
- 尺寸:163.1 x 75.5 x 8.2 mm

相機:
- 後置:50MP + 12MP + 8MP
- 前置:32MP

連線:
- 5G、4G LTE
- Wi-Fi 6、藍牙 5.3
- NFC、GPS
- USB-C

作業系統:
- Android 14 或 iOS 17
- 承諾 4 年系統更新
- 5 年安全性更新
8.4 聯絡客服